In [ ]:
import demjson
import re

In [1]:
def ObjectId(s):
    return s
def extract_jjw():
    with open('jjw_people.json','r') as f:
        yield from map(lambda line:eval(line),f)

In [4]:
def get_info(people):
    '''
        return: 
        {
            'cn': [(title, subject)],
            'en': []
        }
    '''
    cn_paper = []
    en_paper = []
    author_subject = people['subject']
    cn_subjects = []
    en_subjects = []
    for paper in people['paper_list']:
        assert paper['titleCn'] is not None or paper['titleEn'] is not None, 'no title'
        if paper['titleCn'] is not None:
            cn_paper.append(paper['titleCn'])
            cn_subjects.append(paper['subject'])
        else:
            en_paper.append(paper['titleEn'])
            en_subjects.append(paper['subject'])
    if author_subject is None:
        for patent in people['patent_list']:
            if 'firstSubject' in patent and patent['firstSubject'] is not None:
                author_subject = patent['firstSubject']
                break
    if author_subject is not None:
        cn_subjects = map(lambda x:author_subject if x is None else x, cn_subjects)
        en_subjects = map(lambda x:author_subject if x is None else x, en_subjects)
    return {
        'cn': list(zip(cn_paper,cn_subjects)),
        'en': list(zip(en_paper,en_subjects))
    }

In [5]:
cn_papers = []
en_papers = []
for paper in map(get_info, extract_jjw()):
    cn_papers += paper['cn']
    en_papers += paper['en']

In [10]:
from collections import Counter
Counter(map(lambda x:x[1],cn_papers))

Counter({None: 309494,
         '管理科学部': 4687,
         '医学科学部': 23148,
         '信息科学部': 11622,
         '数理科学部': 8653,
         '化学科学部': 5357,
         '生命科学部': 10914,
         '工程与材料科学部': 20599,
         '地球科学部': 2934})

In [7]:
len(en_papers)

403812

In [ ]:
next(jjw_iter)

In [12]:
def paper_iter(people):
    yield from people['paper_list']
    
for paper in map(paper_iter,extract_jjw()):
    if 'firstSubject' in paper:
        print(paper['firstSubject'])

In [15]:
cn_papers_labled = filter(lambda x:x[1] is not None,cn_papers)
import LTP
cn_papers_cut = map(lambda x:'%s\t%s\n'%(' '.join(LTP.cut(x[0])),x[1]),cn_papers_labled)
with open('jjw_labeled.tsv','w') as f:
    f.writelines(cn_papers_cut)

In [81]:
import torchtext
from torchtext import data
jjw_path = 'jjw_labeled.tsv'
text = data.Field(sequential=True, lower=True, batch_first=True)
label = data.Field(sequential=False, use_vocab=False,
                   batch_first=True, is_target=True)
jjw_data = data.TabularDataset(path=jjw_path,
                               format='tsv',
                               fields=[('text', text), ('label', label)])
text.build_vocab(jjw_data, min_freq=20)
label.build_vocab(jjw_data)

In [79]:
labels.build_vocab(jjw_data)

In [65]:
batch_iter = data.Iterator(jjw_data,batch_size=64)
next(iter(batch_iter)).text

tensor([[44750, 13159,    31,  ...,     6, 11828,   390],
        [  756,  3113,  7781,  ...,   682, 12862,    62],
        [ 2143,   683,    11,  ..., 48492,  8573, 40769],
        ...,
        [    1,     1,     1,  ...,  3855,     1,     1],
        [    1,     1,     1,  ...,   488,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])

In [69]:
import torch

In [83]:
text.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             '的': 2,
             '研究': 3,
             '与': 4,
             '对': 5,
             '基于': 6,
             '中': 7,
             '分析': 8,
             '及': 9,
             '影响': 10,
             '和': 11,
             '在': 12,
             '应用': 13,
             '方法': 14,
             '系统': 15,
             '及其': 16,
             '进展': 17,
             '性能': 18,
             '模型': 19,
             '治疗': 20,
             '细胞': 21,
             '设计': 22,
             '作用': 23,
             '技术': 24,
             '一': 25,
             '特性': 26,
             '结构': 27,
             '-': 28,
             '、': 29,
             '种': 30,
             '多': 31,
             '下': 32,
             '算法': 33,
             '/': 34,
             ')': 35,
             '(': 36,
             '临床': 37,
             '表达': 38,
             '例': 39,
             '基因': 40,
             '优化'

In [84]:
import pickle
with open('titles_cut.pkl','rb') as f:
    titles_cut = pickle.load(f)


In [92]:
title = data.Field(sequential=True,lower=True,batch_first=True)
title_ex = data.Example.fromlist(titles_cut,('title',title))
title_data = data.Dataset(title_ex,('title',title),num_worker=10)

ValueError: too many values to unpack (expected 2)

In [96]:
titles_cut = list(map(lambda x:data.Example.fromlist(x,('title',title)),titles_cut))

ValueError: too many values to unpack (expected 2)

In [98]:
with open('wanfang_titles.tsv','w') as f:
    f.writelines('\n'.join(titles_cut))